In [37]:
import pandas as pd
import numpy as np
import pandas as pd
from docxtpl import DocxTemplate,  InlineImage
from docx.shared import Mm
import sys
import mysql.connector
import os
from dotenv import load_dotenv
from datetime import date
# id=sys.argv[1]
id=82
today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host=DB_HOST,
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
cotizacion=pd.read_sql("select * from quotations where id ="+str(id),cnx)
cliente=pd.read_sql("""select * from customers where customers.id="""+str(cotizacion['customer_id'].values[0]),cnx)
user=pd.read_sql("""select * from users where users.id="""+str(cotizacion['user_id'].values[0]),cnx)
productos=pd.read_sql("""select * from cart_products where cart_products.quotation_id="""+str(cotizacion['id'].values[0]),cnx)
questionario=pd.read_sql("""select * from questionaries where questionaries.quotation_id="""+str(cotizacion['id'].values[0]),cnx)
if(len(questionario)==0):
    questionario=pd.read_sql("""select * from questionaries limit 1""",cnx)

import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")

from tablas_dict import tablas
from tablas_dict import redact
aceros=pd.read_sql('select * from steels ',cnx)
aceros.loc[aceros['caliber']=='EST 3 IN','caliber']='EST3'

products=pd.DataFrame()
#iterar sobre tablas
for i in tablas:
    
    #buscar en la base de datos todos los productos de esta tabla
    #pertenecientes a la cotizacion pedida por el usuario.
    p=pd.read_sql('select * from '+i+' where quotation_id = '+str(id),cnx)
    p=p.assign(tabla=i)
    if(('cost' not in p.columns)&(len(p)>0)):
        if('caliber' not in p.columns):
             #esto es en especifico por un caso en que todas kas piezas son cal 14
             p=p.assign(caliber='14')
        try:
            p['caliber']=p['caliber'].str.replace('-','')
        except:
            print(' ')
        costo=aceros.loc[aceros['caliber']==str(p['caliber'].values[0]),'cost'].values[0]
        if('total_kg' in p.columns):
            p=p.assign(cost=costo*p.total_kg)
        if('total_weight' in p.columns):
           
            p=p.assign(cost=costo*p.total_weight)
        if('weight_kg' in p.columns):
         
            p=p.assign(cost=costo*p.weight_kg)
        if('weight' in p.columns):
          
            p=p.assign(cost=costo*p.weight)
        if('long' in p.columns):
           
            p=p.assign(cost=costo*p.long)
        
    products=products.append(p,ignore_index=True)
cols_to_fill_str=['description','protector','model','sku']
products[cols_to_fill_str]=products[cols_to_fill_str].fillna('')
cols_kg=['weight','total_kg','total_weight','weight_kg']
cols_m2=['m2','total_m2']
largo_cols=['long','length','length_meters','frame_background',
       'length_dimension', 'dimensions']
ancho_cols=['uncut_front',  'uncut_background',
       'depth']
products[cols_kg+cols_m2+largo_cols+ancho_cols]=products[cols_kg+cols_m2+largo_cols+ancho_cols].fillna(0)
price_cols=['price','total_price','import','unit_price']
products[price_cols] =products[price_cols].fillna(0)    
pricelist_protectors=pd.read_sql('select * from price_list_protectors',cnx)
quotation_protectors=pd.read_sql('select quotation_protectors.*, protectors.sku from quotation_protectors  inner join protectors on protectors.protector=quotation_protectors.protector where quotation_id ='+str(id),cnx)
quotation_shlf=pd.read_sql('select * from selective_heavy_load_frames where quotation_id ='+str(id),cnx)
materials=pd.read_sql('select * from (materials left join price_list_screws on materials.price_list_screw_id= price_list_screws.id)left join price_lists on price_lists.id=materials.price_list_id',cnx)
materials['type']=materials['type'].fillna('')

doc = DocxTemplate("plantilla.docx")

instalacion_tables=['quotation_installs','quotation_uninstalls']
productos=[]
print(products.columns)
for i in range(len(products)):
    this_color=' '
    seccion=np.nan
    altura=0
    ancho=0
    if(products['tabla'].values[i] not in instalacion_tables):
        if(products['tabla'].values[i]=='selective_heavy_load_frames'):
            this_color='Azul'
            seccion=questionario['section'].values[0]
        if('joist' in products['tabla'].values[i]):
            this_color='Anaranjado'
        
        productos.append({'nombre':redact[products['tabla'].values[i]],
                        'precio':products[price_cols].sum(axis=1).values[i],
                        'cantidad':products['amount'].values[i],
                        'color': this_color,
                        'largo': products[largo_cols].sum(axis=1).values[i],
                        
                        'altura': products[largo_cols].sum(axis=1).values[i],
                        'ancho': products[ancho_cols].sum(axis=1).values[i],
                        'depth': products['depth'].values[i],
                        'model': products['model'].values[i],
                        'seccion':seccion})
                                  
precio_total=products[price_cols].sum(axis=1).sum()
kilos_totales=products[cols_kg].sum(axis=1).sum()
fletes_tables=['packagings','quotation_travel_assignments']
instalacion_tables=['quotation_installs','quotation_uninstalls']
precios=products[price_cols+['tabla','print']]
costo_flete=precios.loc[precios['tabla'].isin(fletes_tables)].sum(axis=1,numeric_only=True).sum()
costo_instalacion=precios.loc[(precios['tabla'].isin(instalacion_tables))&(precios['print']=='Sí')].sum(axis=1,numeric_only=True).sum()
costo_instalacion_incluida=precios.loc[(precios['tabla'].isin(instalacion_tables))&(precios['print']=='In')].sum(axis=1,numeric_only=True).sum()

text=str(questionario['ndib'].values[0])+','
dibujos=[]
while(',' in text):
    print('entra iteracion')
    myindex=text.index(',')
    print(myindex)
    dibujos.append(text[0:myindex])
    text=text[myindex+1:]

if(len(dibujos)>0):
    primer_dibujo=dibujos[0]
else:
    primer_dibujo=' '
context={
    'cliente':cliente['customer'].values[0],
    'direccion':cliente['address'].values[0]+' '+cliente['outdoor'].values[0]+', '+cliente['city'].values[0]+' '+cliente['suburb'].values[0]+' '+cliente['state'].values[0]+', cp: '+str(cliente['zip_code'].values[0]),
    'folio': cotizacion['invoice'].values[0],
    'fecha': today,
    'asesor': user['name'].values[0],
    'productos': productos,
    'precio_total': precio_total,
    'kilos_totales': kilos_totales,
    'costo_flete':costo_flete,
    'costo_instalacion':costo_instalacion,
    'costo_instalacion_incluida':costo_instalacion_incluida,
    'costo_selectivo':precio_total - costo_flete -costo_instalacion,
    'estado': cliente['state'].values[0],
    'a5': questionario['a5'].values[0], #que productos e almacena
   #reativos de nivel
    'a8': questionario['a8'].values[0],#frente
    'a9': questionario['a9'].values[0],#fondo
    'a10': questionario['a10'].values[0],#alto
    'a11': questionario['a11'].values[0],#peso
# dimensiones de la tarima
    'a18': questionario['a18'].values[0],#frente
    'a19': questionario['a19'].values[0],#fondo
    'a20': questionario['a20'].values[0],#alto
    'a21': questionario['a21'].values[0],#peso
#ambiente
    'a25': questionario['a25'].values[0], #temperatura
    'a26': questionario['a26'].values[0], #inflamable
    'a27': questionario['a27'].values[0], #explosivo
    'a28': questionario['a28'].values[0], #corrosivo
#Reactivos extra
#     'ndib':  str(questionario['ndib'].values[0]).replace(',',"""
# """), #numero de dibujos
'ndib':  str(questionario['ndib'].values[0]).replace(',','O'),
'primer_dibujo':primer_dibujo ,
    'npos':  questionario['npos'].values[0], #numero de posiciones
    'vigas':  questionario['vigas'].values[0], #vigas
    'tiempo':  questionario['tiempo'].values[0], #tiempo de entrega
    'dibujos': dibujos,
    }
doc.render(context) 
doc.save("storage/Cotizacion"+str(id)+".docx")


C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cotizacion=pd.read_sql("select * from quotations where id ="+str(id),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cliente=pd.read_sql("""select * from customers where customers.id="""+str(cotizacion['customer_id'].values[0]),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user=p

C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(id),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or s

C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(id),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  p=pd.read_sql('select * from '+i+' where quotation_id = '+str(id),cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_19020\2151743404.py:81: FutureWarni

Index(['id', 'quotation_id', 'amount', 'type', 'depth', 'developing', 'long',
       'caliber', 'kg_m2', 'weight', 'm2', 'connector', 'sku', 'unit_price',
       'total_price', 'created_at', 'updated_at', 'tabla', 'length', 'camber',
       'height', 'model', 'total_load_kg', 'total_poles', 'total_crossbars',
       'total_diagonals', 'total_plates', 'total_kg', 'total_m2', 'skate',
       'loading_capacity', 'type_joist', 'length_meters', 'weight_kg', 'use',
       'destination', 'front', 'color', 'background', 'cost', 'dimensions',
       'joist_type', 'price', 'state', 'unit', 'import', 'frame_background',
       'length_dimension', 'total_weight', 'price_unit', 'description',
       'system', 'f_total', 'print', 'breakdown_install', 'protector',
       'breakdown_uninstall', 'uncut_front', 'cut_front', 'uncut_background',
       'cut_background'],
      dtype='object')
entra iteracion
4


In [38]:
productos

[{'nombre': 'Parrilla Métalica',
  'precio': 0.0,
  'cantidad': 1,
  'color': ' ',
  'largo': 0.0,
  'ancho': 0.0,
  'depth': 0.0,
  'model': ''},
 {'nombre': 'FLETE',
  'precio': 3.42,
  'cantidad': 1,
  'color': ' ',
  'largo': 0.0,
  'ancho': 0.0,
  'depth': 0.0,
  'model': ''},
 {'nombre': 'FLETE',
  'precio': 6.84,
  'cantidad': 2,
  'color': ' ',
  'largo': 0.0,
  'ancho': 0.0,
  'depth': 0.0,
  'model': ''},
 {'nombre': 'Panel metálico Galvanizado modelo ',
  'precio': 910.23,
  'cantidad': 1,
  'color': ' ',
  'largo': 1.7469999999999999,
  'ancho': 0.0,
  'depth': 0.0,
  'model': ''},
 {'nombre': 'Panel metálico Pintado modelo',
  'precio': 1820.45,
  'cantidad': 2,
  'color': ' ',
  'largo': 1.7469999999999999,
  'ancho': 0.0,
  'depth': 0.0,
  'model': ''},
 {'nombre': 'Panel metálico Galvanizado modelo',
  'precio': 725.71,
  'cantidad': 1,
  'color': ' ',
  'largo': 1.7730000000000001,
  'ancho': 0.0,
  'depth': 0.0,
  'model': ''},
 {'nombre': 'Panel metálico Pintado mode

In [25]:
fletes_tables=['packagings','quotation_travel_assignments']
instalacion_tables=['quotation_installs','quotation_uninstalls']
precios=products[price_cols+['tabla']]
costo_flete=precios.loc[precios['tabla'].isin(fletes_tables)].sum(axis=1,numeric_only=True).sum()
costo_instalacion=precios.loc[precios['tabla'].isin(instalacion_tables)].sum(axis=1,numeric_only=True).sum()


In [7]:
import pandas as pd
import numpy as np
import pandas as pd
from docxtpl import DocxTemplate,  InlineImage
from docx.shared import Mm
import sys
import mysql.connector
import os
from dotenv import load_dotenv
from datetime import date
# id=sys.argv[1]
id=82
today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = 'tyrsaadmin'
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host=DB_HOST,
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
cursor=cnx.cursor(buffered=True)

In [2]:
data=pd.read_excel('CATALOGOPROVEEDORES.xlsx')

In [24]:
data=data.fillna(0)

In [3]:
data.columns

Index(['NUMERO DE PROVEEDOR', 'Estatus', 'RAZON SOCIAL', 'RFC', 'NOMBRE CORTO',
       'Calle', 'Número interior', 'Número exterior', 'Entre calle 1',
       'Colonia', 'Código postal', 'Población', 'Municipio', 'Estado',
       'Nacionalidad', 'Teléfono', 'Clasificación', 'CURP', 'Forma de pago',
       'Beneficiario', 'Titular', 'Banco', 'Sucursal (Banco)',
       'Num. de cuenta', 'Observación', 'Mail predeterminado',
       'Régimen fiscal'],
      dtype='object')

In [27]:
query="""insert into proovedores(customer, customer_rfc, customer_state, customer_city,customer_suburb, customer_street, customer_outdoor,customer_indoor, customer_zip_code, customer_email,customer_telephone, legal_name,alias,clave, regimen_fiscal, curp,nacionalidad) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
for i in range(len(data)):
    cursor.execute(query,tuple([
        data['RAZON SOCIAL'].values[i],
        data['RFC'].values[i],
        data['Estado'].values[i],
        data['Población'].values[i],
        data['Municipio'].values[i],
        data['Calle'].values[i],
        str(data['Número exterior'].values[i]),
        str(data['Número interior'].values[i]),
        str(data['Código postal'].values[i]),
        data['Mail predeterminado'].values[i],
        str(data['Teléfono'].values[i]),
        data['RAZON SOCIAL'].values[i],
        data['NOMBRE CORTO'].values[i],
        str(data['NUMERO DE PROVEEDOR'].values[i]),
        data['Régimen fiscal'].values[i],
        data['CURP'].values[i],
        data['Nacionalidad'].values[i],]
    ))

In [8]:
colums=pd.read_sql('select * from customers limit 1',cnx).columns

C:\Users\hp\AppData\Local\Temp\ipykernel_19020\77830236.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  colums=pd.read_sql('select * from customers limit 1',cnx).columns


In [28]:
cnx.commit()